# Merging restaurant inspection data and community area data

We found Census data and public health data for each community area. We also found crimes data and sanitation complaints data included community area. Wanting to connect data from these datasets to our restaurant inspection dataset, we used the crime dataset to obtain the community areas of the restaurants (this is explained below). 

We aggregated the number of crimes and sanitation complaints by community area. We then merged these two aggregations as well as the Census and public health data with the restaurant inspection data.

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import math
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn import discriminant_analysis
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn import tree
from sklearn import ensemble
from sklearn.cross_validation import KFold
import csv
import StringIO

In [2]:
# load full data; drop observations with no location
full = pd.read_csv('full data.csv')
full = full.iloc[:, :-1]
full_drop_no_loc = full.dropna(subset = [full.columns[-4]])
full_drop_no_loc.columns

Index([u'Unnamed: 0', u'Inspection ID', u'DBA Name', u'AKA Name', u'License #',
       u'Facility Type', u'Risk', u'Address', u'City', u'State', u'Zip',
       u'Inspection Date', u'Inspection Type', u'Results', u'Violations',
       u'Latitude', u'Longitude', u'Location', u'Day of Week', u'Month'],
      dtype='object')

We dropped restaurant inspections with no location because without location data, there was no way to connect Census or crime data to the restaurant inspections data frame. We did not feel that it was reasonable to attempt to impute missing values for the location of a restaurant. Luckily, there were not that many restaurant inspections with missing locations.

In [3]:
# load crimes data; drop crimes with no location
crimes = pd.read_csv('Crimes_2015.csv')
print crimes.columns
crimes_drop_no_loc = crimes.dropna(subset = [crimes.columns[-1]])

Index([u'ID', u'Case Number', u'Date', u'Block', u'IUCR', u'Primary Type',
       u'Description', u'Location Description', u'Arrest', u'Domestic',
       u'Beat', u'District', u'Ward', u'Community Area', u'FBI Code',
       u'X Coordinate', u'Y Coordinate', u'Year', u'Updated On', u'Latitude',
       u'Longitude', u'Location'],
      dtype='object')


Without crime location, there is no way to connect it to a particular restaurant or group of restaurants, so we dropped each crime for which the location was missing.

In [4]:
# load complaints data; drop complaints with no location
complaints = pd.read_csv('Complaints.csv')
print complaints.columns
complaints_drop_no_loc = complaints.dropna(subset = [complaints.columns[-1]])

Index([u'Creation Date', u'Status', u'Completion Date',
       u'Service Request Number', u'Type of Service Request',
       u'What is the Nature of this Code Violation?', u'Street Address',
       u'ZIP Code', u'X Coordinate', u'Y Coordinate', u'Ward',
       u'Police District', u'Community Area', u'Latitude', u'Longitude',
       u'Location'],
      dtype='object')


Without complaint location, there is no way to connect it to a particular restaurant or group of restaurants, so we dropped each sanitation complaint for which the location was missing.

In [5]:
# getting community area of restaurants using crime data
crimes_knn = KNeighborsClassifier(n_neighbors = 1)
crimes_knn.fit(crimes_drop_no_loc.iloc[:, (-3, -2)], crimes_drop_no_loc['Community Area'])
full_comm_area_pred = crimes_knn.predict(full_drop_no_loc.iloc[:, (-5, -4)])
full_comm_area_pred_str = [str(int(i)) for i in full_comm_area_pred]

full_comm_area = full_drop_no_loc.copy()
full_comm_area['Community Area'] = pd.Series(full_comm_area_pred_str, index = full_comm_area.index)

Both the crime data and the restaurant inspection data contained latitude and longitude variables, but only the crime data contained community area. Because there was existing Census and public health data for the community areas of Chicago, we wanted to determine the community area of each restaurant to connect the Census and public health data to the restaurant inspection data. Because of the number of crimes and the number of restaurants, for each restaurant, there was some crime that occurred very close by. Thus, we assigned each restaurant the community area of the closest crime. 

In [8]:
# getting # crimes in each community area
crimes_comm_area = np.unique(crimes_drop_no_loc['Community Area'])
crimes_comm_area_str = [str(int(i)) for i in crimes_comm_area]

num_crimes_comm_area = []
for i in crimes_comm_area:
    num_crimes_comm_area.append(len(crimes_drop_no_loc[crimes_drop_no_loc['Community Area'] == i]))

num_crimes_comm_area_df = pd.DataFrame(data = {'Community Area' : crimes_comm_area_str, \
                                       'num_crimes_comm_area' : num_crimes_comm_area})

We will use number of crimes in each community area in 2015 as a covariate for our model.

In [9]:
# getting # complaints in each community area
complaints_comm_area = np.unique(complaints_drop_no_loc['Community Area'])
complaints_comm_area_str = [str(int(i)) for i in complaints_comm_area]

num_complaints_comm_area = []
for i in complaints_comm_area:
    num_complaints_comm_area.append(len(complaints_drop_no_loc[complaints_drop_no_loc['Community Area'] == i]))

num_complaints_comm_area_df = pd.DataFrame(data = {'Community Area' : complaints_comm_area_str, \
                                       'num_complaints_comm_area' : num_complaints_comm_area})

We will use number of sanitation complaints each in community area since 2011 as a covariate for our model.

In [19]:
# load census data; change column name for community area number
census_chi = pd.read_csv('Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012.csv')
census_chi.columns = ['Community Area', 'COMMUNITY AREA NAME',
       'PERCENT OF HOUSING CROWDED', 'PERCENT HOUSEHOLDS BELOW POVERTY', 'PERCENT AGED 16+ UNEMPLOYED',
       'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA', 'PERCENT AGED UNDER 18 OR OVER 64', 'PER CAPITA INCOME ','HARDSHIP INDEX']
print census_chi.columns

for i in range(len(census_chi) - 1):
    census_chi.iloc[i, 0] = str(int(census_chi.iloc[i, 0]))

Index([u'Community Area', u'COMMUNITY AREA NAME',
       u'PERCENT OF HOUSING CROWDED', u'PERCENT HOUSEHOLDS BELOW POVERTY',
       u'PERCENT AGED 16+ UNEMPLOYED',
       u'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA',
       u'PERCENT AGED UNDER 18 OR OVER 64', u'PER CAPITA INCOME ',
       u'HARDSHIP INDEX'],
      dtype='object')


We found Census data for Chicago for the years 2008-2012 that included selected socioeconomic indicators for each community area. We plan to use these indicators as covariates in our model.

In [20]:
# load public health data; delete data that we already have from census; delete columns with missing data
pub_health_chi = pd.read_csv('Public_Health_Statistics-_Selected_public_health_indicators_by_Chicago_community_area.csv')
pub_health_chi = pub_health_chi.iloc[:, :-3]
wanted_cols = [i for i in range(2, len(pub_health_chi.columns) - 8)]
wanted_cols.append(0)
wanted_cols.append(22)
wanted_cols.append(25)
wanted_cols.sort()
pub_health_chi = pub_health_chi.iloc[:, (wanted_cols)]
print pub_health_chi.columns

for i in range(len(pub_health_chi)):
    pub_health_chi.iloc[i, 0] = str(int(pub_health_chi.iloc[i, 0]))

Index([u'Community Area', u'Birth Rate', u'General Fertility Rate',
       u'Low Birth Weight', u'Prenatal Care Beginning in First Trimester',
       u'Preterm Births', u'Teen Birth Rate', u'Assault (Homicide)',
       u'Breast cancer in females', u'Cancer (All Sites)',
       u'Colorectal Cancer', u'Diabetes-related', u'Firearm-related',
       u'Infant Mortality Rate', u'Lung Cancer', u'Prostate Cancer in Males',
       u'Stroke (Cerebrovascular Disease)', u'Tuberculosis', u'Dependency'],
      dtype='object')


We found public health data Chicago for the years 2005-2011 that included selected health indicators for each community area. We plan to use these indicators as covariates in our model. Some of the indicators were repeats from the Census data, so those indicators were deleted.

In [21]:
# merge complaints and crimes by community area
comm_area_df = pd.merge(num_crimes_comm_area_df, num_complaints_comm_area_df, how = 'left', on = ['Community Area']) 

In [22]:
# merge census data with crimes and complaints by community area
comm_area_df = pd.merge(comm_area_df, census_chi, how = 'left', on = ['Community Area']) 

In [23]:
# merge public health data with census, crimes, and complaints  data by community area
comm_area_df = pd.merge(comm_area_df, pub_health_chi, how = 'left', on = ['Community Area']) 

In [24]:
# delete community area 0 (doesn't exist)
comm_area_df = comm_area_df.iloc[1:, :]

In [25]:
# merge restaurant inspection and community area data
full_data = pd.merge(full_comm_area, comm_area_df, how = 'left', on = ['Community Area'])
full_data.columns

Index([u'Unnamed: 0', u'Inspection ID', u'DBA Name', u'AKA Name', u'License #',
       u'Facility Type', u'Risk', u'Address', u'City', u'State', u'Zip',
       u'Inspection Date', u'Inspection Type', u'Results', u'Violations',
       u'Latitude', u'Longitude', u'Location', u'Day of Week', u'Month',
       u'Community Area', u'num_crimes_comm_area', u'num_complaints_comm_area',
       u'COMMUNITY AREA NAME', u'PERCENT OF HOUSING CROWDED',
       u'PERCENT HOUSEHOLDS BELOW POVERTY', u'PERCENT AGED 16+ UNEMPLOYED',
       u'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA',
       u'PERCENT AGED UNDER 18 OR OVER 64', u'PER CAPITA INCOME ',
       u'HARDSHIP INDEX', u'Birth Rate', u'General Fertility Rate',
       u'Low Birth Weight', u'Prenatal Care Beginning in First Trimester',
       u'Preterm Births', u'Teen Birth Rate', u'Assault (Homicide)',
       u'Breast cancer in females', u'Cancer (All Sites)',
       u'Colorectal Cancer', u'Diabetes-related', u'Firearm-related',
       u'Infant Mo

In [26]:
# combining with restaurant inspection data that doesn't have community area
wanted_cols = [0]
for i in range(len(full_data.columns)):
    if i >= 20:
        wanted_cols.append(i)
all_full = pd.merge(full, full_data.iloc[:, (wanted_cols)], how = 'left', on = ['Unnamed: 0'])
all_full.columns

Index([u'Unnamed: 0', u'Inspection ID', u'DBA Name', u'AKA Name', u'License #',
       u'Facility Type', u'Risk', u'Address', u'City', u'State', u'Zip',
       u'Inspection Date', u'Inspection Type', u'Results', u'Violations',
       u'Latitude', u'Longitude', u'Location', u'Day of Week', u'Month',
       u'Community Area', u'num_crimes_comm_area', u'num_complaints_comm_area',
       u'COMMUNITY AREA NAME', u'PERCENT OF HOUSING CROWDED',
       u'PERCENT HOUSEHOLDS BELOW POVERTY', u'PERCENT AGED 16+ UNEMPLOYED',
       u'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA',
       u'PERCENT AGED UNDER 18 OR OVER 64', u'PER CAPITA INCOME ',
       u'HARDSHIP INDEX', u'Birth Rate', u'General Fertility Rate',
       u'Low Birth Weight', u'Prenatal Care Beginning in First Trimester',
       u'Preterm Births', u'Teen Birth Rate', u'Assault (Homicide)',
       u'Breast cancer in females', u'Cancer (All Sites)',
       u'Colorectal Cancer', u'Diabetes-related', u'Firearm-related',
       u'Infant Mo

In [27]:
# write file with restaurant inspection data and community area data added
with open('full_data_2.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerow(['Unnamed: 0', 'Inspection ID', 'DBA Name', 'AKA Name', 'License #',
       'Facility Type', 'Risk', 'Address', 'City', 'State', 'Zip',
       'Inspection Date', 'Inspection Type', 'Results', 'Violations',
       'Latitude', 'Longitude', 'Location', 'Day of Week', 'Month',
       'Community Area', 'num_crimes_comm_area', 'num_complaints_comm_area',
       'COMMUNITY AREA NAME', 'PERCENT OF HOUSING CROWDED',
       'PERCENT HOUSEHOLDS BELOW POVERTY', 'PERCENT AGED 16+ UNEMPLOYED',
       'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA',
       'PERCENT AGED UNDER 18 OR OVER 64', 'PER CAPITA INCOME ',
       'HARDSHIP INDEX', 'Birth Rate', 'General Fertility Rate',
       'Low Birth Weight', 'Prenatal Care Beginning in First Trimester',
       'Preterm Births', 'Teen Birth Rate', 'Assault (Homicide)',
       'Breast cancer in females', 'Cancer (All Sites)',
       'Colorectal Cancer', 'Diabetes-related', 'Firearm-related',
       'Infant Mortality Rate', 'Lung Cancer', 'Prostate Cancer in Males',
       'Stroke (Cerebrovascular Disease)', 'Tuberculosis', 'Dependency'])
    writer.writerows(all_full.values)

In [28]:
# data file with community area data only
with open('comm_area_data.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerow(['Community Area', 'num_crimes_comm_area', 'num_complaints_comm_area',
       'COMMUNITY AREA NAME', 'PERCENT OF HOUSING CROWDED',
       'PERCENT HOUSEHOLDS BELOW POVERTY', 'PERCENT AGED 16+ UNEMPLOYED',
       'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA',
       'PERCENT AGED UNDER 18 OR OVER 64', 'PER CAPITA INCOME ',
       'HARDSHIP INDEX', 'Birth Rate', 'General Fertility Rate',
       'Low Birth Weight', 'Prenatal Care Beginning in First Trimester',
       'Preterm Births', 'Teen Birth Rate', 'Assault (Homicide)',
       'Breast cancer in females', 'Cancer (All Sites)',
       'Colorectal Cancer', 'Diabetes-related', 'Firearm-related',
       'Infant Mortality Rate', 'Lung Cancer', 'Prostate Cancer in Males',
       'Stroke (Cerebrovascular Disease)', 'Tuberculosis', 'Dependency'])
    writer.writerows(comm_area_df.values)